In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random 
import torch as T
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset, ConcatDataset, IterableDataset
import numpy as np
import pandas as pd

import math
import argparse
import import_ipynb
from bgan_i_ganLambda import *
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

from argparse import ArgumentParser

save_path = "data/saved_models/ganWganLambda.tar"#vaegan_model - Copy.tar"
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

if not os.path.exists("data/saved_models"):
    os.makedirs("data/saved_models")

importing Jupyter notebook from bgan_i_ganLambda.ipynb
importing Jupyter notebook from ugan_i_ganLambda.ipynb


In [2]:
ARG_PARSER = ArgumentParser()

ARG_PARSER.add_argument('--nfeatures', default=609, type=int)
ARG_PARSER.add_argument('--dfeatures', default=43, type=int)
ARG_PARSER.add_argument('--ehidden', default=300, type=int)
ARG_PARSER.add_argument('--model', type=str)

ARG_PARSER.add_argument('--air', default=True)
ARG_PARSER.add_argument('--mimic', default=False)

ARG_PARSER.add_argument('--num_epochs', default=200, type=int)
ARG_PARSER.add_argument('--seq_len', default=20, type=int)
ARG_PARSER.add_argument('--pred_len', default=8, type=int)
ARG_PARSER.add_argument('--missingRate', default=10, type=int)
ARG_PARSER.add_argument('--patience', default=200, type=int)
ARG_PARSER.add_argument('--e_lrn_rate', default=0.1, type=float)
ARG_PARSER.add_argument('--g_lrn_rate', default=0.1, type=float)
ARG_PARSER.add_argument('--d_lrn_rate', default=0.001, type=float)
ARG_PARSER.add_argument('--resume_training', default=False)
ARG_PARSER.add_argument('--train', default=False)
ARG_PARSER.add_argument('--evalImp', default=True)
ARG_PARSER.add_argument('--evalPred', default=False)

_StoreAction(option_strings=['--eval'], dest='eval', nargs=None, const=None, default=True, type=None, choices=None, help=None, metavar=None)

In [3]:
ARGS = ARG_PARSER.parse_args(args=[])
MAX_SEQ_LEN = ARGS.seq_len
BATCH_SIZE = ARGS.batch_size
EPSILON = 1e-40

In [5]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf#11.1179
        self.delta = delta

    def __call__(self, val_loss, model, discriminator, optimizer, optimizer_d, save_path):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, discriminator, optimizer, optimizer_d, save_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, discriminator, optimizer, optimizer_d, save_path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, discriminator, optimizer, optimizer_d, save_path):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        T.save({
            "model": model.state_dict(),
            'trainer': optimizer.state_dict(),
            "discriminator": discriminator.state_dict(),
            'trainer_d': optimizer_d.state_dict()
        }, save_path)
        self.val_loss_min = val_loss

In [6]:
def pred_test(args, model,discriminator,predWin):
    model.eval()
    
    RLoss=0
    FLoss=0
    mseLoss=0
    mseLossF=0
    TBatches=0
    oBmi=[]
    oBmiF=[]
    iBmi=[]
    oAge=[]
    oSex=[]
    imputations=[]
    with T.autograd.no_grad():
        if args.air:
            data=pd.read_csv('.../aaai/data/air/preprocess/airTest.csv',header=0)
            mask=pd.read_csv('.../aaai/data/air/preprocess/airTestMask.csv',header=0)
            data=data[['Date', 'Time', 'Month', 'PT08.S1(CO)', 'CO(GT)', 'NMHC(GT)','C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)','PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']]
            del data['Time']
            del data['Date']
            del mask['Time']
            del mask['Date']

        if args.mimic:
            data=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTest.csv',header=0)
            mask=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTestMask.csv',header=0)
            del data['subject_id']
            del data['charttime']
            del mask['subject_id']
            del mask['charttime']
            print(data.shape)
            data=data[['ALBUMIN', 'ANION GAP','WBC', 'BANDS', 'BICARBONATE', 'BILIRUBIN', 'BUN',
           'CHLORIDE', 'CREATININE', 'GLUCOSE', 'HEMATOCRIT', 'HEMOGLOBIN', 'INR',
           'LACTATE', 'PaCO2', 'PLATELET', 'POTASSIUM', 'PT', 'PTT', 'SODIUM'
           ]]

        
        data = T.as_tensor(data.values.astype(float), dtype=T.float32)
        data=data.view(int(data.shape[0]/args.seq_len), args.seq_len, data.shape[1])
        
        mask = T.as_tensor(mask.values.astype(float), dtype=T.float32)
        mask=mask.view(int(mask.shape[0]/args.seq_len), args.seq_len, mask.shape[1])
        
        loss={}

        decay=mask[:,:,1]
        rdecay=mask[:,:,2]
        mask=mask[:,:,0]


        data=data.squeeze()
        mask=mask.squeeze()
        decay=decay.squeeze()
        rdecay=rdecay.squeeze()
        #print(data.shape)
        #print(mask.shape)
        #print(decay.shape)

        #values to be predicted
        y = data.clone().detach()
        testMask = mask.clone().detach()
        
        y=y[:,:,2]
       

        #------------remove last 5 timestamps------------------
        #print(data[0:10,8:,653])
        for i in range(data.shape[0]):
            #if(data[i,])
            j=20
            if(predWin==8):
                k=16
                decay[i,j-k:j]=T.tensor([0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8])
                rdecay[i,j-k:j]=T.tensor([8,7.5,7,6.5,6,5.5,5,4.5,4,3.5,3,2.5,2,1.5,1,0.5])
            elif(predWin==7):
                k=14
                decay[i,j-k:j]=T.tensor([0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7])
                rdecay[i,j-k:j]=T.tensor([7,6.5,6,5.5,5,4.5,4,3.5,3,2.5,2,1.5,1,0.5])*120
            elif(predWin==6):
                k=12
                decay[i,j-k:j]=T.tensor([0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6])
                rdecay[i,j-k:j]=T.tensor([6,5.5,5,4.5,4,3.5,3,2.5,2,1.5,1,0.5])
            elif(predWin==5):
                k=10
                decay[i,j-k:j]=T.tensor([0.5,1,1.5,2,2.5,3,3.5,4,4.5,5])
                rdecay[i,j-k:j]=T.tensor([5,4.5,4,3.5,3,2.5,2,1.5,1,0.5])

            data[i,j-k:j,:]=0
            mask[i,j-k:j]=0
            y[i,0:j-k]=0
            #print(sex.shape,age.shape)
            #yOrig[i,0:j-k]=0
            testMask[i,0:j-k]=0
            
            #print(data[i,j-k:j,0])
            #print(mask[i,0:j-k,0])


        ret_f, ret,disc = run_on_batch(model,discriminator,data,mask,decay,rdecay, args, optimizer=None,optimizer_d=None,epoch=None)#,bmi_norm)
        print("Input",data.shape)
        print(data[0,:,2])
        print("Reverse",ret['imputations'][0,:])
        print("ForwardOnly",ret_f['imputations'][0,:])
        print("Original",y.shape)
        print(y[0,:])
        print("Mask",testMask.shape)
        print(testMask[0,:])
        RLoss=RLoss+ret['loss']
        FLoss=FLoss+ret_f['loss']
        testMask=testMask.cuda()
        y=y.cuda()
        outputBMI=ret['imputations'] * testMask
        outputBMIF=ret_f['imputations'] * testMask
        mseLoss=mseLoss+ (torch.sum(torch.abs(outputBMI-y)))/ (torch.sum(testMask) + 1e-5)
        mseLossF=mseLossF+ (torch.sum(torch.abs(outputBMIF-y)))/ (torch.sum(testMask) + 1e-5)
#         print("RMSELoss Revrese: ",mseLoss)
#         print("RMSELoss Forward: ",mseLossF)
        outBmi, outBmiF,inBmi = plotBmi(outputBMI, outputBMIF, y, testMask)
        oBmi.extend(outBmi)
        oBmiF.extend(outBmiF)
        iBmi.extend(inBmi)

        
        RLoss = RLoss
        mseLoss = mseLoss
        mseLossF = mseLossF
    #print("===================================")
    oBmi=np.asarray(oBmi)
    iBmi=np.asarray(iBmi)
    loss = (oBmi - iBmi)
    loss=np.asarray([abs(number) for number in loss])
    variance = sum([((x - mseLoss) ** 2) for x in loss]) / len(loss) 
    res = variance ** 0.5
    ci=1.96*(res/(math.sqrt(len(loss))))

    #print("Val R Loss:",RLoss)
    print("CI",ci)
    print("MAE Loss Reverse:",mseLoss)
    #print("MAE Loss Forward:",mseLossF)
    #print(outputBMI)
    return oBmi,oBmiF,iBmi
                
                

In [7]:
def imputation_test(args, model,discriminator,missingRate):
    model.eval()
    
    RLoss=0
    FLoss=0
    mseLoss=0
    mseLossF=0
    TBatches=0
    oBmi=[]
    oBmiF=[]
    iBmi=[]
    oAge=[]
    oSex=[]
    imputations=[]
    samples=0
    pids=0
    with T.autograd.no_grad():
        if args.air:
            data=pd.read_csv('.../aaai/data/air/preprocess/airTest.csv',header=0)
            mask=pd.read_csv('.../aaai/data/air/preprocess/airTestMask.csv',header=0)
            data=data[['Date', 'Time', 'Month', 'PT08.S1(CO)', 'CO(GT)', 'NMHC(GT)','C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)','PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']]
            del data['Time']
            del data['Date']
            del mask['Time']
            del mask['Date']

        if args.mimic:
            data=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTest.csv',header=0)
            mask=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTestMask.csv',header=0)
            del data['subject_id']
            del data['charttime']
            del mask['subject_id']
            del mask['charttime']
            print(data.shape)
            data=data[['ALBUMIN', 'ANION GAP','WBC', 'BANDS', 'BICARBONATE', 'BILIRUBIN', 'BUN',
           'CHLORIDE', 'CREATININE', 'GLUCOSE', 'HEMATOCRIT', 'HEMOGLOBIN', 'INR',
           'LACTATE', 'PaCO2', 'PLATELET', 'POTASSIUM', 'PT', 'PTT', 'SODIUM'
           ]]

        
        data = T.as_tensor(data.values.astype(float), dtype=T.float32)
        data=data.view(int(data.shape[0]/args.seq_len), args.seq_len, data.shape[1])
        
        mask = T.as_tensor(mask.values.astype(float), dtype=T.float32)
        mask=mask.view(int(mask.shape[0]/args.seq_len), args.seq_len, mask.shape[1])
        
        loss={}

        decay=mask[:,:,1]
        rdecay=mask[:,:,2]
        mask=mask[:,:,0]

        data=data.squeeze()
        mask=mask.squeeze()
        decay=decay.squeeze()
        rdecay=rdecay.squeeze()
        #print(data.shape)
        #print(mask.shape)
        #print(decay.shape)

        #values to be predicted
        y = data.clone().detach()
        testMask = mask.clone().detach()
        y=y[:,:,2]

        #------------remove last 5 timestamps------------------
        #print(data[0:10,8:,653])
        for i in range(data.shape[0]):
            #if(data[i,])
            j=20
            k=16
            #mask[i,:].loc[mask[i,:].query('value == 1').sample(frac=.1).index,'value'] = 0
            idxs = torch.nonzero(mask[i,:] == 1)
            #print("idxs",list(idxs.size())[0])
            #print(idxs)
            samples=samples+list(idxs.size())[0]
            if(missingRate==50):
                if(list(idxs.size())[0]>4):
                    idxs=random.sample(set(idxs),5)
                    data[i,idxs[0],2]=0
                    data[i,idxs[1],2]=0
                    data[i,idxs[2],2]=0
                    data[i,idxs[3],2]=0
                    data[i,idxs[4],2]=0
                    #print(mask[i,:])
                    mask[i,idxs[0]]=0
                    mask[i,idxs[1]]=0
                    mask[i,idxs[2]]=0
                    mask[i,idxs[3]]=0
                    mask[i,idxs[4]]=0
                    pids=pids + 5
                    break;
            if(missingRate>=40):
                if(list(idxs.size())[0]>3):
                    idxs=random.sample(set(idxs),4)
                    data[i,idxs[0],2]=0
                    data[i,idxs[1],2]=0
                    data[i,idxs[2],2]=0
                    data[i,idxs[3],2]=0
                    #print(mask[i,:])
                    mask[i,idxs[0]]=0
                    mask[i,idxs[1]]=0
                    mask[i,idxs[2]]=0
                    mask[i,idxs[3]]=0
                    pids=pids + 4
                    break;
            if(missingRate>=30):
                if(list(idxs.size())[0]>2):
                    idxs=random.sample(set(idxs),3)
                    data[i,idxs[0],2]=0
                    data[i,idxs[1],2]=0
                    data[i,idxs[2],2]=0
                    #print(mask[i,:])
                    mask[i,idxs[0]]=0
                    mask[i,idxs[1]]=0
                    mask[i,idxs[2]]=0
                    pids=pids + 3
                    break;
            if(missingRate>=20):
                if(list(idxs.size())[0]>1):
                    idxs=random.sample(set(idxs),2)
                    data[i,idxs[0],2]=0
                    data[i,idxs[1],2]=0
                    #print(mask[i,:])
                    mask[i,idxs[0]]=0
                    mask[i,idxs[1]]=0
                    pids=pids + 2
                    break;
            if(missingRate>=10):
                if(list(idxs.size())[0]>0):
                    idxs=random.sample(set(idxs),1)
                    data[i,idxs,2]=0
                    mask[i,idxs]=0
                    pids=pids + 1
            #print(idxs)
            #print("---------------")
            #remove values from last 50% of the sequence


            #print(mask[i,:])
            testMask[i,:]=testMask[i,:]-mask[i,:]
            y[i,:]=y[i,:]*testMask[i,:]


        #print("Input Data",data.shape)
        #print("Input Mask",mask.shape)
        ret_f,ret,disc = run_on_batch(model,discriminator,data,mask,decay,rdecay, args, optimizer=None,optimizer_d=None,epoch=None)#,bmi_norm)
#             print("Input",data.shape)
#             print(data[0,:,316])
#             print("Reverse",ret['imputations'][0,:])
#             print("ForwardOnly",ret_f['imputations'][0,:])
#             print("Original",y.shape)
#             print(y[0,:])
        #print("Mask",testMask.shape)
        #print(testMask[0,:])
        RLoss=RLoss+ret['loss']
        FLoss=FLoss+ret_f['loss']
        testMask=testMask.cuda()
        y=y.cuda()
        #print("Output",ret['imputations'].shape)
        #print("Output",ret_f['imputations'].shape)
        #print("Output mask",testMask.shape)
        outputBMI=ret['imputations'] * testMask
        outputBMIF=ret_f['imputations'] * testMask
        #print("outputBMI",outputBMI.shape)
        #print(outputBMI[0])
        #print("outputBMIF",outputBMIF.shape)
        #print(outputBMIF[0])
        mseLoss=mseLoss+ (torch.sum(torch.abs(outputBMI-y)))/ (torch.sum(testMask) + 1e-5)
        mseLossF=mseLossF+ (torch.sum(torch.abs(outputBMIF-y)))/ (torch.sum(testMask) + 1e-5)
#             print("RMSELoss Revrese: ",mseLoss)
#             print("RMSELoss Forward: ",mseLossF)
        outBmi, outBmiF,inBmi = plotBmi(outputBMI, outputBMIF, y, testMask)
        oBmi.extend(outBmi)
        oBmiF.extend(outBmiF)
        iBmi.extend(inBmi)
        

        RLoss = RLoss
        mseLoss = mseLoss
        mseLossF = mseLossF
    #print("===================================")
    oBmi=np.asarray(oBmi)
    iBmi=np.asarray(iBmi)
    loss = (oBmi - iBmi)
    loss=np.asarray([abs(number) for number in loss])
    variance = sum([((x - mseLoss) ** 2) for x in loss]) / len(loss) 
    res = variance ** 0.5
    ci=1.96*(res/(math.sqrt(len(loss))))

    #print("Val R Loss:",RLoss)
    print("CI",ci)
    print("MAE Loss Reverse:",mseLoss)
    #print("MSE Loss Forward:",mseLossF)
    #print(outputBMI)
    return oBmi,oBmiF,iBmi
                

In [8]:
def plotBmi(outBmi,outBmiF, inBmi, testMask):
    
    outBmi = outBmi.cpu().detach().numpy()
    outBmiF = outBmiF.cpu().detach().numpy()
    inBmi = inBmi.cpu().detach().numpy()
#     outAge = outAge.cpu().detach().numpy()
#     outSex = outSex.cpu().detach().numpy()
    testMask = testMask.cpu().detach().numpy()
    
   
    outBmi=outBmi[np.nonzero(testMask)]
    outBmiF=outBmiF[np.nonzero(testMask)]
    inBmi=inBmi[np.nonzero(testMask)]
#     outAge=outAge[np.nonzero(testMask)]
#     outSex=outSex[np.nonzero(testMask)]
    
    return outBmi,outBmiF,inBmi#, outAge, outSex

In [9]:
def run_evalFull(args, model, discriminator):
    model.eval()
    
    RLoss=0
    GLoss=0
    DLoss=0
    TBatches=0
    oBmi=[]
    iBmi=[]
    oAge=[]
    oSex=[]
   
    with T.autograd.no_grad():
        if args.air:
            data=pd.read_csv('.../aaai/data/air/preprocess/airVal.csv',header=0)
            mask=pd.read_csv('.../aaai/data/air/preprocess/airValMask.csv',header=0)
            data=data[['Date', 'Time', 'Month', 'PT08.S1(CO)', 'CO(GT)', 'NMHC(GT)','C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)','PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']]
            del data['Time']
            del data['Date']
            del mask['Time']
            del mask['Date']

        if args.mimic:
            data=pd.read_csv('.../aaai/data/mimic/preprocess/mimicVal.csv',header=0)
            mask=pd.read_csv('.../aaai/data/mimic/preprocess/mimicValMask.csv',header=0)
            del data['subject_id']
            del data['charttime']
            del mask['subject_id']
            del mask['charttime']
            print(data.shape)
            data=data[['ALBUMIN', 'ANION GAP','WBC', 'BANDS', 'BICARBONATE', 'BILIRUBIN', 'BUN',
           'CHLORIDE', 'CREATININE', 'GLUCOSE', 'HEMATOCRIT', 'HEMOGLOBIN', 'INR',
           'LACTATE', 'PaCO2', 'PLATELET', 'POTASSIUM', 'PT', 'PTT', 'SODIUM'
           ]]

        
        
        data = T.as_tensor(data.values.astype(float), dtype=T.float32)
        data=data.view(int(data.shape[0]/args.seq_len), args.seq_len, data.shape[1])
        
        mask = T.as_tensor(mask.values.astype(float), dtype=T.float32)
        mask=mask.view(int(mask.shape[0]/args.seq_len), args.seq_len, mask.shape[1])

        
        loss={}

        decay=mask[:,:,1]
        rdecay=mask[:,:,2]
        mask=mask[:,:,0]

        data=data.squeeze()
        mask=mask.squeeze()
        decay=decay.squeeze()
        rdecay=rdecay.squeeze()
        #print(data.shape)
        #print(mask.shape)
        #print(decay.shape)


        ret_f, ret, disc = run_on_batch(model,discriminator,data,mask,decay,rdecay, args, optimizer=None,optimizer_d=None,epoch=None)#,bmi_norm)
        RLoss=RLoss+ret['loss']
        GLoss=GLoss+disc['loss_g'].item()
        DLoss=DLoss+disc['loss_d'].item()

        RLoss = RLoss
        GLoss=GLoss
        DLoss=DLoss
    #print("===================================")
    print("Val R Loss:",RLoss)
    print("Val G Loss:",GLoss)
    print("Val D Loss:",DLoss)
    #print(outputBMI)
    return RLoss,GLoss,DLoss
                
                

In [10]:
def run_epoch(args, model, discriminator):
    ''' Run a single epoch
    '''

    trainLoss=[]
    discLoss=[]
    gLoss=[]
    valLoss=[]
    gValLoss=[]
    discValLoss=[]
    
    #define optimizer
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)
    optimizer_d = optim.Adam(discriminator.parameters(), lr = 1e-3)
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    if args.resume_training:
        checkpoint = T.load(save_path)
        optimizer.load_state_dict(checkpoint['trainer'])
        optimizer_d.load_state_dict(checkpoint['trainer_d'])
        early_stopping(1.896514, model, discriminator, optimizer, optimizer_d, save_path)
    
    #for evrey epoch
    for epoch in range(args.num_epochs):
        model.train()
    
        #Running Losses
        RLoss=0
        GLoss=0
        DLoss=0
        TBatches=0  
        print("=============EPOCH=================")
        
        if args.air:
            data=pd.read_csv('.../aaai/data/air/preprocess/airTrain.csv',header=0)
            mask=pd.read_csv('.../aaai/data/air/preprocess/airTrainMask.csv',header=0)
            data=data[['Date', 'Time', 'Month', 'PT08.S1(CO)', 'CO(GT)', 'NMHC(GT)','C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)','PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']]
            #data=data[['Date', 'Time', 'Month', 'PT08.S1(CO)', 'NO2(GT)','CO(GT)', 'NMHC(GT)','C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)','PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']]
            del data['Time']
            del data['Date']
            del mask['Time']
            del mask['Date']

        if args.mimic:
            data=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTrain.csv',header=0)
            mask=pd.read_csv('.../aaai/data/mimic/preprocess/mimicTrainMask.csv',header=0)
            del data['subject_id']
            del data['charttime']
            del mask['subject_id']
            del mask['charttime']
            print(data.shape)
            data=data[['ALBUMIN', 'ANION GAP','WBC', 'BANDS', 'BICARBONATE', 'BILIRUBIN', 'BUN',
           'CHLORIDE', 'CREATININE', 'GLUCOSE', 'HEMATOCRIT', 'HEMOGLOBIN', 'INR',
           'LACTATE', 'PaCO2', 'PLATELET', 'POTASSIUM', 'PT', 'PTT', 'SODIUM'
           ]]

        
        
        data = T.as_tensor(data.values.astype(float), dtype=T.float32)
        data=data.view(int(data.shape[0]/args.seq_len), args.seq_len, data.shape[1])
        
        mask = T.as_tensor(mask.values.astype(float), dtype=T.float32)
        mask=mask.view(int(mask.shape[0]/args.seq_len), args.seq_len, mask.shape[1])
        
#         print(data.shape)
#         print(mask.shape)
        #print(decay.shape)

        decay=mask[:,:,1]
        rdecay=mask[:,:,2]
        mask=mask[:,:,0]

        data=data.squeeze()
        mask=mask.squeeze()
        decay=decay.squeeze()
        rdecay=rdecay.squeeze()
        
#         print(data.shape)
#         print(mask.shape)
#         print(decay.shape)


        ret_f, ret, disc = run_on_batch(model,discriminator,data,mask,decay,rdecay, args, optimizer,optimizer_d,epoch)#,bmi_norm)
        RLoss=RLoss+ret['loss'].item()
        GLoss=GLoss+disc['loss_g'].item()
        DLoss=DLoss+disc['loss_d'].item()

       
        RLoss=RLoss
        GLoss=GLoss
        DLoss=DLoss
        
        print("EPOCH:", epoch, "loss_R:", "%.4f"%RLoss, "loss_G:", "%.4f"%GLoss, "loss_D:", "%.4f"%DLoss)
        
        trainLoss.append(RLoss)
        discLoss.append(DLoss)
        gLoss.append(GLoss)

        valid_loss,g_valloss,disc_loss = run_evalFull(args, model,discriminator)
        
        valLoss.append(valid_loss)
        discValLoss.append(disc_loss)
        gValLoss.append(g_valloss)
        #plotBmi(outBmi , inBmi)

        
        #if epoch<1 or epoch >5:
        if not (T.isnan(valid_loss)):
            early_stopping(valid_loss, model, discriminator, optimizer, optimizer_d, save_path)

        if early_stopping.early_stop:
            print("Early stopping")
            break

        #plot_grad_flow(model['e'].named_parameters())
        #plot_grad_flow(model['g'].named_parameters())
        #plot_grad_flow(model['d'].named_parameters())
    return trainLoss,discLoss,gLoss, valLoss,discValLoss,gValLoss


In [11]:
def run(args):
    
    train_on_gpu = T.cuda.is_available()
    if train_on_gpu:
        print('Training on GPU.')
    else:
        print('No GPU available, training on CPU.')
        
    model = BGAN(args)
    discriminator=BGAN_D()
    
    #print("discriminator",discriminator)

    if torch.cuda.is_available():
        model = model.cuda()
        discriminator=discriminator.cuda()

    if args.resume_training:
        checkpoint = T.load(save_path)
        model.load_state_dict(checkpoint['model'])
        discriminator.load_state_dict(checkpoint['discriminator'])
        #optimizer = optim.Adam(model.parameters(), lr = 1e-3)
        #optimizer.load_state_dict(checkpoint['trainer'])
        trainLoss,discLoss,gLoss, valLoss,discValLoss,gValLoss = run_epoch(args, model, discriminator) 
        #return trainLoss,discLoss,gLoss, valLoss,discValLoss,gValLoss
    
    elif args.train:
        trainLoss,discLoss,gLoss, valLoss,discValLoss,gValLoss = run_epoch(args, model, discriminator) 
        #return trainLoss,discLoss,gLoss, valLoss,discValLoss,gValLoss
        
    elif args.evalImp:
        #load Model
        checkpoint = T.load(save_path)
        model.load_state_dict(checkpoint['model'])
        discriminator.load_state_dict(checkpoint['discriminator'])
        optimizer = optim.Adam(model.parameters(), lr = 1e-3)
        optimizer.load_state_dict(checkpoint['trainer'])
        optimizer_d = optim.Adam(discriminator.parameters(), lr = 1e-3)
        optimizer_d.load_state_dict(checkpoint['trainer_d'])
        #RLoss,GLoss,DLoss = run_evalFull(args, model, discriminator)
        oBmi, oBmiF, iBmi, oAge,oSex = imputation_test(args, model,discriminator,args.missingRate)
        
    elif args.evalPred:
        #load Model
        checkpoint = T.load(save_path)
        model.load_state_dict(checkpoint['model'])
        discriminator.load_state_dict(checkpoint['discriminator'])
        optimizer = optim.Adam(model.parameters(), lr = 1e-3)
        optimizer.load_state_dict(checkpoint['trainer'])
        optimizer_d = optim.Adam(discriminator.parameters(), lr = 1e-3)
        optimizer_d.load_state_dict(checkpoint['trainer_d'])
        #RLoss,GLoss,DLoss = run_evalFull(args, model, discriminator)
        oBmi, oBmiF, iBmi, oAge,oSex = pred_test(args, model,discriminator,args.pred_len)
        
        #return oBmi, oBmiF, iBmi, oAge,oSex

In [12]:
#trainLoss,discLoss,gLoss, valLoss,discValLoss,gValLoss = run(ARGS)
#RLoss,GLoss,DLoss = run(ARGS)
#oBmi, oBmiF, iBmi, oAge, oSex = run(ARGS)

run(ARGS)

Training on GPU.
Val R Loss: tensor(61.0351, device='cuda:0')
MSE Loss Reverse: tensor(3.9483, device='cuda:0')
MSE Loss Forward: tensor(1.9724, device='cuda:0')
